In [2]:
import random
import numpy as np
import tensorflow as tf

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from sktime.datasets import load_UCR_UEA_dataset

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [5]:
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed);

In [4]:
X_train, y_train = load_UCR_UEA_dataset(name="Strawberry",
                                        split='train', 
                                        return_type='numpy3D',
                                        extract_path="./data")
X_test, y_test = load_UCR_UEA_dataset(name="Strawberry",
                                      split='test', 
                                      return_type='numpy3D',
                                      extract_path='./data')

In [6]:
class DNNTSClassifier(nn.Module):
    def __init__(self, input_shape, num_classes):
        super().__init__()
        self.flatten = nn.Flatten()
        
        # Calculate input size for the first dense layer
        n_timesteps, n_features = input_shape
        input_size = n_timesteps * n_features
        
        self.layers = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.3),
            
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.3),
            
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.3),
            
            nn.Linear(64, num_classes)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        x = self.layers(x)
        return x

In [7]:
def train_model(model, train_loader, val_loader, device, epochs=100, patience=10):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())
    
    best_val_loss = float('inf')
    best_model = None
    patience_counter = 0
    
    for epoch in range(epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                val_loss += loss.item()
                
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == batch_y).sum().item()
                total += batch_y.size(0)
        
        val_loss /= len(val_loader)
        val_accuracy = correct / total
        
        print(f'Epoch {epoch+1}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')
        
        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print('Early stopping triggered')
                break
    
    # Load best model
    model.load_state_dict(best_model)
    return model

In [8]:
def predict(model, X, device):
    model.eval()
    X_tensor = torch.FloatTensor(X).to(device)
    
    with torch.no_grad():
        outputs = model(X_tensor)
        _, predicted = torch.max(outputs, 1)
    
    return predicted.cpu().numpy()

In [17]:
y_train = y_train.astype(int) - 1
y_test = y_test.astype(int) - 1

In [18]:
X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train, 
    y_train,
    test_size=0.2,
    random_state=42
)

In [19]:
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.LongTensor(y)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create datasets
train_dataset = TimeSeriesDataset(X_train_final, y_train_final)
val_dataset = TimeSeriesDataset(X_val, y_val)
    
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


In [21]:
model = DNNTSClassifier(
    input_shape=(X_train.shape[1], X_train.shape[2]),
    num_classes=len(np.unique(y_train))
).to(device)

In [22]:
model = train_model(model, train_loader, val_loader, device)

Epoch 1, Val Loss: 0.6768, Val Accuracy: 0.5447
Epoch 2, Val Loss: 0.7350, Val Accuracy: 0.6098
Epoch 3, Val Loss: 0.2393, Val Accuracy: 0.9106
Epoch 4, Val Loss: 0.3170, Val Accuracy: 0.8455
Epoch 5, Val Loss: 0.1758, Val Accuracy: 0.9431
Epoch 6, Val Loss: 0.2110, Val Accuracy: 0.9268
Epoch 7, Val Loss: 0.1477, Val Accuracy: 0.9350
Epoch 8, Val Loss: 0.1464, Val Accuracy: 0.9268
Epoch 9, Val Loss: 0.5850, Val Accuracy: 0.8537
Epoch 10, Val Loss: 0.2398, Val Accuracy: 0.9024
Epoch 11, Val Loss: 0.4347, Val Accuracy: 0.8618
Epoch 12, Val Loss: 0.2897, Val Accuracy: 0.9106
Epoch 13, Val Loss: 0.3547, Val Accuracy: 0.9024
Epoch 14, Val Loss: 0.1280, Val Accuracy: 0.9431
Epoch 15, Val Loss: 0.2886, Val Accuracy: 0.9106
Epoch 16, Val Loss: 0.2856, Val Accuracy: 0.9187
Epoch 17, Val Loss: 0.1099, Val Accuracy: 0.9593
Epoch 18, Val Loss: 0.1504, Val Accuracy: 0.9350
Epoch 19, Val Loss: 0.0987, Val Accuracy: 0.9675
Epoch 20, Val Loss: 0.1072, Val Accuracy: 0.9431
Epoch 21, Val Loss: 0.0814, V

In [23]:
y_pred = predict(model, X_test, device)
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [24]:
clf_report = classification_report(y_test, y_pred)

print(clf_report)

              precision    recall  f1-score   support

           0       0.89      0.98      0.93       132
           1       0.99      0.93      0.96       238

    accuracy                           0.95       370
   macro avg       0.94      0.96      0.95       370
weighted avg       0.95      0.95      0.95       370

